# Benchmarks of Strategies for Selecting Outdated Items
This notebook contains the benchmarks related to the selection strategies for context data, which we report in our paper.
Context data are selected from the processed data and included in the next progressive computation step, such that its result approximates that of a _non-progressive_ computation over the processed data.

## Benchmark Configuration

We use the following configuration in our benchmarks:
### Test cases 
- full computation over the entire dataset (upper baseline)
- progressive computation without optimization (lower baseline)
- full computation of processed data
- progressive computation using optimization strategies

### Dataset
- NYC taxis dataset (10 Million items), stored in a compressed CSV file, loaded with DuckDB 

### Variables
- dependent variables: runtime, prediction error
- independent variables: 

## Benchmarks

### Finding an appropriate `chunk size`
The number of items in each chunk dictate the computation time for each chunk in the data, in that the more items we process, the longer the DOI computation takes.
Therefore, the first consideration in our benchmarks is to find the maximum number of items, for which the computation time remains immediate.
Prior work (see Card et al., 1991) has shown this limit to be about one second.

In the cell below, we try different chunk sizes to find the maximum items we can pass to the doi function for computations under 1s.

In [8]:
from time import time
from copy import copy
from demo_test_case import DATA, DOI_CONFIG, PARAMETERS, PATH
from context_item_selection_strategy.no_context import NoContext
from outdated_item_selection_strategy.no_update import NoUpdate
from storage_strategy.no_storage import NoStorage
from test_case import create_test_case, StrategiesConfiguration

chunk_sizes = [10, 100, 1000, 10000]

for size in chunk_sizes:
  before = time()

  strategies_config = StrategiesConfiguration(
    "__chunk_size__",
    NoContext(DATA.n_dims, None), 
    NoUpdate(DATA.n_dims, None), 
    NoStorage(),
  )

  parameter_config = copy(PARAMETERS)
  parameter_config.chunks = 1
  parameter_config.chunk_size = size
  
  create_test_case(
    name="__test__", 
    strategies=strategies_config,
    data=DATA,
    doi=DOI_CONFIG,
    params=parameter_config,
    path=PATH,
  ).run()

  print(f"{size}: {time() - before}s")


10: 0.11207771301269531s
100: 0.1100459098815918s
1000: 0.24966645240783691s
10000: 1.8293428421020508s


### Computing the Baselines
#### Baseline1: Monolithic computation
The ground truth for our strategies is a full computation over the entire dataset without any chunking.
This computation naturally takes a long time to complete, which is why the progressive scenario is so much more effective from a user perspective: we get to see the data much faster.

In the context of the `BenchmarkTestCase` class, the monolithic computation corresponds with running a progression with a single chunk. 

#### Baseline2: Bigger chunks
In addition to the full computation, another important baseline is to compare ourselves against a computation that does not use any strategies, but instead uses the entire `chunk time` to compute a whole new chunk.
The idea here is to compare, whether all the context- and outdated-computations are actually valuable, or whether we could just use all resources on processing new data instead.

In [ ]:
from demo_test_case import *
from test_case import create_ground_truth_test_case, create_bigger_chunks_test_case

create_ground_truth_test_case(DATA, DOI_CONFIG, PARAMETERS, PATH).run()
create_bigger_chunks_test_case(DATA, DOI_CONFIG, PARAMETERS, PATH).run()

### Running the test cases

In [ ]:
from demo_test_case import *
from test_case import create_test_case, get_short_title, StrategiesConfiguration
from storage_strategy.windowing_storage import WindowingStorage

n_test_cases = len(context_strategies) * len(update_strategies)
i = 0


print(f"data: {data_label}, {get_short_title(DOI_CONFIG, PARAMETERS)}\n####")
for c_strat in context_strategies:
  for u_strat in update_strategies:
    i += 1
    print(f"({i}/{n_test_cases}) context: {c_strat[0]}, update: {u_strat[0]}")

    # check if already completed
    name = f"{c_strat[0]}-{u_strat[0]}"
    if os.path.isfile(f"{PATH}/doi/{name}.csv") and os.path.isfile(f"{PATH}/times/{name}.csv"):
      print("skipping test case because already completed.")
      continue

    strategy_config = StrategiesConfiguration(
      name=name,
      context_strategy=c_strat[1](),
      update_strategy=u_strat[1](),
      storage_strategy=WindowingStorage(max_size=PARAMETERS.max_age*PARAMETERS.chunk_size)
    )
    
    tc = create_test_case(
      name=name,
      strategies=strategy_config,
      data=DATA,
      doi=DOI_CONFIG, 
      params=PARAMETERS,
      path=PATH
    )
    tc.run()

    print(f"done: {tc.pipeline.total_time}s")

### Evaluation

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import jaccard_score, r2_score

rus = RandomUnderSampler(random_state=0)

def evaluate_test_case(test_case: np.ndarray, ground_truth: np.ndarray):
  # score = jaccard_score(test_case, ground_truth, average="weighted")
  score = r2_score(ground_truth, test_case)
  return score

ground_truth = results_full["doi"]
context_test_case = results_context["doi"]
baseline_test_case = results_chunked["doi"]


evaluate_test_case(baseline_test_case, ground_truth), evaluate_test_case(context_test_case, ground_truth)